In [1]:
import gdspy
import pandas as pd
import numpy as np
import sys
import math
import pattern_writer as pw

In [2]:
baseUnit = 1 #layout scale(microns)
m=baseUnit*1e6 
nm = m*1E-9

In [3]:
'''def layout_placement(basisX, basisY, numberX, numberY, generator_function, placement_function):
    Xs=[]
    Ys=[]
    lastX=0
    lastY=0
    while len(Xs)< numberX:
        Xs.append(lastX)
        lastX+=basisX
    while len(Ys)< numberY:
        Ys.append(lastY)
        lastY+=basisY'''

'def layout_placement(basisX, basisY, numberX, numberY, generator_function, placement_function):\n    Xs=[]\n    Ys=[]\n    lastX=0\n    lastY=0\n    while len(Xs)< numberX:\n        Xs.append(lastX)\n        lastX+=basisX\n    while len(Ys)< numberY:\n        Ys.append(lastY)\n        lastY+=basisY'

In [4]:
def layout_placement(basisX, basisY, numberX, numberY, generator_function, placement_function):
    array=[]
    lastX=0
    for nX in range(numberX):
        lastY=0
        for nY in range(numberY):
            geometry_cell= generator_function(placement_function(lastX, lastY))
            array.append(gdspy.CellReference(geometry_cell, (lastX, lastY), magnification=1))
            #pw.array_singleReference(lastX, lastY, geometry_cell)
            lastY+=basisY
        lastX+=basisX
    return array

In [5]:
#geometry1 = gdspy.Cell("CIRCLE")
#geometry1.add(gdspy.Round((0, 0), 93*nm, tolerance=0.001))
#geometry2 = gdspy.Cell("OVAL")
#geometry2.add(gdspy.Round((0, 0), [93*nm, 80*nm], tolerance=1e-4))
#def p_f(x,y):
 #   return np.sqrt(x**2 + y**2)
#def g_f(distance):
 #   if distance < 200*nm:
        #geometry1 = gdspy.Cell("CIRCLE")
        #geometry1.add(gdspy.Round((0, 0), 93*nm, tolerance=0.001))
  #      return geometry1
   # else:
        #geometry2 = gdspy.Cell("OVAL")
        #geometry2.add(gdspy.Round((0, 0), [93*nm, 80*nm], tolerance=1e-4))
    #    return geometry2
#y = layout_placement(300*nm,100*nm,2,3,g_f, p_f)

In [6]:
#y

In [7]:
def layout_placement2(basisX, basisY, angle, numberX, numberY, generator_function, placement_function):
    array=[]
    array_X=[]
    array_Y=[]
    lastX=0
    theta = np.radians(angle)
    c, s = np.cos(theta), np.sin(theta)
    R = np.array(((c, -s), (s, c)))
    for nX in range(numberX):
        array_X.append(lastX)
        lastY=0
        for nY in range(numberY):
            if len(array_Y)<numberY:
                array_Y.append(lastY)
            new_point= np.matmul(R,(lastX,lastY))
            geometry_cell= generator_function(placement_function(new_point))
            array.append(gdspy.CellReference(geometry_cell, (new_point), magnification=1))
            #pw.array_singleReference(lastX, lastY, geometry_cell)
            lastY+=(2*basisY)
        lastX+=(2*basisX)
    array2_X= np.add(array_X, basisX).tolist()
    array2_Y= np.add(array_Y, basisY).tolist()
    #print(array2_Y)
    for n2X in array2_X:
        for n2Y in array2_Y:
            new_point = np.matmul(R,(n2X,n2Y))
            geometry_cell= generator_function(placement_function(new_point))
            array.append(gdspy.CellReference(geometry_cell, (new_point), magnification=1))
    return array

In [8]:
def layout_placement3(basis1, basis2, numberX, numberY, generator_function, placement_function):
    array=[]
    array_X=[]
    array_Y=[]
    lastX=0
    for nX in range(numberX):
        array_X.append(lastX)
        lastY=0
        for nY in range(numberY):
            if len(array_Y)<numberY:
                array_Y.append(lastY)
            geometry_cell= generator_function(placement_function(lastX, lastY))
            array.append(gdspy.CellReference(geometry_cell, (lastX, lastY), magnification=1))
            #pw.array_singleReference(lastX, lastY, geometry_cell)
            lastY+=(basis1[1])
        lastX+=(basis1[0])
    print(array_X)
    print(array_Y)
    array2_X= np.add(array_X, basis2[0]).tolist()
    print(array2_X)
    array2_Y= np.add(array_Y, basis2[1]).tolist()
    print(array2_Y)
    #print(array2_Y)
    for n2X in array2_X:
        for n2Y in array2_Y:
            geometry_cell= generator_function(placement_function(n2X, n2Y))
            array.append(gdspy.CellReference(geometry_cell, (n2X, n2Y), magnification=1))
    return array

In [9]:
#geometry1 = gdspy.Cell("CIRCLE")
#geometry1.add(gdspy.Round((0, 0), 250*nm, tolerance=0.001))
#geometry2 = gdspy.Cell("OVAL")
#geometry2.add(gdspy.Round((0, 0), [250*nm, 233*nm], tolerance=1e-4))
#def p_f(x,y):
    #return np.sqrt(x**2 + y**2)
#def g_f(distance):
    #if distance < 500*nm:
        #geometry1 = gdspy.Cell("CIRCLE")
        #geometry1.add(gdspy.Round((0, 0), 93*nm, tolerance=0.001))
        #return geometry1
    #else:
        #geometry2 = gdspy.Cell("OVAL")
        #geometry2.add(gdspy.Round((0, 0), [93*nm, 80*nm], tolerance=1e-4))
        #return geometry2
#y = layout_placement3([300*nm,(np.sqrt(3))*100*nm],[150*nm,100*nm],5,8,g_f, p_f)

In [10]:
#y

In [11]:
def posArrayGenerator(a, b, numA, numB, xlimit, ylimit, generator_function, placement_function):
    arr=[]
    new_x=0
    new_y=0
    for x in range(-numA,numA):
        for y in range(-numB,numB):
            vec=[(x*a[0]+y*b[0], x*a[1]+y*b[1])]
            if vec[0][0]<= xlimit and vec[0][0]>= -xlimit and vec[0][1] <= ylimit and vec[0][1] >= -ylimit:
                new_x= vec[0][0]
                new_y= vec[0][1]
            if vec[0][0]>= xlimit:
                new_x=vec[0][0]- (2*xlimit)
                #geometry_cell= generator_function(placement_function(new_x, vec[0][1]))
                #arr.append(gdspy.CellReference(geometry_cell, (new_x, vec[0][1]), magnification=1))
            elif vec[0][0]<= -xlimit:
                new_x=vec[0][0]+ (2*xlimit)
                #geometry_cell= generator_function(placement_function(new_x, vec[0][1]))
                #arr.append(gdspy.CellReference(geometry_cell, (new_x, vec[0][1]), magnification=1))
            if vec[0][1]>= ylimit:
                new_y=vec[0][1]- (2*ylimit)
                #geometry_cell= generator_function(placement_function(vec[0][0], new_y))
                #arr.append(gdspy.CellReference(geometry_cell, (vec[0][0], new_y), magnification=1))
            elif vec[0][1]<= -ylimit:
                new_y=vec[0][1]+ (2*ylimit)
                #geometry_cell= generator_function(placement_function(vec[0][0], new_y))
                #arr.append(gdspy.CellReference(geometry_cell, (vec[0][0], new_y), magnification=1))
            geometry_cell= generator_function(placement_function(new_x, new_y))
            arr.append(gdspy.CellReference(geometry_cell, (new_x, new_y), magnification=1))
    return arr

In [12]:
def layout_placement4(basis1, basis2, numberX, numberY, generator_function, placement_function):
    line1=[]
    xs=[]
    new_xs=[0,]
    ys=[]
    new_ys=[0,]
    array=[]
    x=0
    y=0
    while x <= numberX and y <= numberY:
        xs.append(x)
        ys.append(y)
        geometry_cell= generator_function(placement_function(x, y))
        array.append(gdspy.CellReference(geometry_cell, (x, y), magnification=1))
        y= y+basis1[1]
        x= x+basis1[0]
    while min(new_xs)<= numberX and min(new_ys) <= numberY:
        new_xs.clear()
        new_ys.clear()
        new_xs= np.add(xs, basis2[0]).tolist()
        #for num in np.add(xs, basis2[0]).tolist():
            #if num not in new_xs and num>=0:
                #new_xs.append(num)
        new_ys= np.add(ys, basis2[1]).tolist()
        #for num in np.add(ys, basis2[1]).tolist():
           # if num not in new_ys and num>=0:
               # new_ys.append(num)
        for x in new_xs:
            for y in new_ys:
                if x in range(0, numberX) and y in range(0, numberY):
                    geometry_cell= generator_function(placement_function(x, y))
                    array.append(gdspy.CellReference(geometry_cell, (x, y), magnification=1))
                else:
                    x = x% numberX
                    y = y% numberY
                    geometry_cell= generator_function(placement_function(x, y))
                    array.append(gdspy.CellReference(geometry_cell, (x, y), magnification=1))
       # xs.clear()
        #ys.clear()
        xs=new_xs.copy()
        ys=new_ys.copy()
    return array

In [13]:
def layout_placement5(N, ratio, generator_function, placement_function):
    arr=[]
    #N= 2345
    #ratio = np.sqrt(3)/2 # cos(60°)
    N_X = int(np.sqrt(N/ratio))
    N_Y = N // N_X
    xv, yv = np.meshgrid(np.arange(N_X), np.arange(N_Y), sparse=False, indexing='xy')
    xv = xv * ratio
    xv[::2, :] += ratio/2
    for nX in xv:
        for nY in yv:
            geometry_cell= generator_function(placement_function((nX, nY))
            arr.append(gdspy.CellReference(geometry_cell, (nX, nY), magnification=1))
    return arr
    

SyntaxError: invalid syntax (<ipython-input-13-99065ab84202>, line 13)

In [ ]:
geometry1 = gdspy.Cell("CIRCLE")
geometry1.add(gdspy.Round((0, 0), 72*nm, tolerance=0.001))
geometry2 = gdspy.Cell("OVAL")
geometry2.add(gdspy.Round((0, 0), [72*nm, 64*nm], tolerance=1e-4))
def p_f(point):
    return np.sqrt(point[0]**2 + point[1]**2)
def g_f(distance):
    if distance < 500*nm:
        #geometry1 = gdspy.Cell("CIRCLE")
        #geometry1.add(gdspy.Round((0, 0), 93*nm, tolerance=0.001))
        return geometry1
    else:
        #geometry2 = gdspy.Cell("OVAL")
        #geometry2.add(gdspy.Round((0, 0), [93*nm, 80*nm], tolerance=1e-4))
        return geometry2
#y = layout_placement2(230*nm, 570*nm,70,4,6,g_f, p_f)
y = layout_placement5(2345, np.sqrt(3)/2,g_f, p_f)

In [ ]:
#mm=[]
#x=posArrayGenerator((150*nm,100*nm),(94*nm,127*nm),10,15, 700*nm, 1100*nm, g_f, p_f)
#x
#for i in range(len(x)):
   # mm.append(gdspy.CellReference(geometry1, (x[i][0], x[i][1]), magnification=1))

In [ ]:
gdspy.current_library = gdspy.GdsLibrary()
Lens = gdspy.Cell("LENS")
Lens.add(y)
gdspy.current_library.add(Lens)
gdspy.write_gds('Testing29.gds')